In [ ]:
!pip install openvino
!pip install openvino-dev

In [ ]:
!git clone https://github.com/openvinotoolkit/training_extensions.git

In [ ]:
%cd training_extensions

In [ ]:
!pip install '.[base]'

In [ ]:
!otx --help

In [ ]:
%cd /content
!mkdir data
%cd data
!git clone https://github.com/thsant/wgisd.git
%cd wgisd
!git checkout 6910edc5ae3aae8c20062941b1641821f0c30127

In [ ]:
# format images folder
!mv data images

# format annotations folder
!mv coco_annotations annotations

# rename annotations to meet *_train.json pattern
!mv annotations/train_bbox_instances.json annotations/instances_train.json
!mv annotations/test_bbox_instances.json annotations/instances_val.json
!cp annotations/instances_val.json annotations/instances_test.json

%cd ../..

In [ ]:
!otx find --task DETECTION --pattern atss

In [ ]:
!ls /usr/local/lib/python3.10/dist-packages/otx/recipe/detection/*

In [ ]:
%%capture
!otx train --config  /usr/local/lib/python3.10/dist-packages/otx/recipe/detection/atss_mobilenetv2.yaml \
                     --data_root data/wgisd \
                     --work_dir otx-workspace \
                     --max_epochs 20 --data.train_subset.batch_size 1


In [ ]:
!otx test --work_dir otx-workspace

/usr/local/lib/python3.10/dist-packages/otx/core/data/entity/utils.py:37: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  pytree._register_pytree_node(  # noqa: SLF001
/usr/local/lib/python3.10/dist-packages/otx/cli/cli.py:281: UserWarning: Load default config from otx-workspace/.latest/train/configs.yaml.
  warn(f"Load default config from {self.cache_dir / 'configs.yaml'}.", stacklevel=0)
/usr/local/lib/python3.10/dist-packages/otx/cli/cli.py:274: UserWarning: Load default checkpoint from otx-workspace/.latest/train/checkpoints/epoch_018.ckpt.
  warn(f"Load default checkpoint from {latest_checkpoint}.", stacklevel=0)
/usr/local/lib/python3.10/dist-packages/otx/cli/cli.py:281: UserWarning: Load default config from otx-workspace/.latest/train/configs.yaml.
  warn(f"Load default config from {self.cache_dir / 'configs.yaml'}.", stacklevel=0)
2024-08-31 09:58:28.881108: I tensorflow/core/util/port.cc

In [ ]:
!otx export --work_dir otx-workspace

/usr/local/lib/python3.10/dist-packages/otx/core/data/entity/utils.py:37: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  pytree._register_pytree_node(  # noqa: SLF001
/usr/local/lib/python3.10/dist-packages/otx/cli/cli.py:281: UserWarning: Load default config from otx-workspace/.latest/train/configs.yaml.
  warn(f"Load default config from {self.cache_dir / 'configs.yaml'}.", stacklevel=0)
/usr/local/lib/python3.10/dist-packages/otx/cli/cli.py:274: UserWarning: Load default checkpoint from otx-workspace/.latest/train/checkpoints/epoch_018.ckpt.
  warn(f"Load default checkpoint from {latest_checkpoint}.", stacklevel=0)
/usr/local/lib/python3.10/dist-packages/otx/cli/cli.py:281: UserWarning: Load default config from otx-workspace/.latest/train/configs.yaml.
  warn(f"Load default config from {self.cache_dir / 'configs.yaml'}.", stacklevel=0)
/usr/local/lib/python3.10/dist-packages/otx/algo/common/lo

In [ ]:
!otx optimize  --work_dir otx-workspace \
                         --checkpoint otx-workspace/20240831_064805/exported_model.xml

In [ ]:
!otx test --work_dir otx-workspace \
                    --checkpoint otx-workspace/20240831_100401/optimized_model.xml \
                    --engine.device cpu

/usr/local/lib/python3.10/dist-packages/otx/core/data/entity/utils.py:37: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  pytree._register_pytree_node(  # noqa: SLF001
/usr/local/lib/python3.10/dist-packages/otx/cli/cli.py:281: UserWarning: Load default config from otx-workspace/.latest/train/configs.yaml.
  warn(f"Load default config from {self.cache_dir / 'configs.yaml'}.", stacklevel=0)
/usr/local/lib/python3.10/dist-packages/otx/cli/cli.py:274: UserWarning: Load default checkpoint from otx-workspace/.latest/train/checkpoints/epoch_018.ckpt.
  warn(f"Load default checkpoint from {latest_checkpoint}.", stacklevel=0)
/usr/local/lib/python3.10/dist-packages/otx/cli/cli.py:281: UserWarning: Load default config from otx-workspace/.latest/train/configs.yaml.
  warn(f"Load default config from {self.cache_dir / 'configs.yaml'}.", stacklevel=0)
2024-08-31 10:05:02.477667: I tensorflow/core/util/port.cc

In [ ]:
!mkdir model
!cp /content/otx-workspace/20240831_065017/optimized_model.bin  model
!cp /content/otx-workspace/20240831_065017/optimized_model.xml  model
!ls model

In [ ]:
model_name = "optimized_model"
model_xml_name = f"{model_name}.xml"
model_bin_name = f"{model_name}.bin"

model_xml_path = "model/" +  model_xml_name
model_xml_path

In [ ]:
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import openvino as ov

# Fetch `notebook_utils` module
import requests

r = requests.get(
    url="https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/latest/utils/notebook_utils.py",
)

open("notebook_utils.py", "w").write(r.text)

from notebook_utils import download_file, device_widget

device = device_widget()

core = ov.Core()
model = core.read_model(model=model_xml_path)
compiled_model = core.compile_model(model=model, device_name=device.value)

output_layer = compiled_model.output(0)

In [ ]:
image_filename = "/content/data/flower_photos/test/dandelion/10043234166_e6dd915111_n.jpg"

# image_filename = "/content/data/flower_photos/test/daisy/100080576_f52e8ee070_n.jpg"
# The MobileNet model expects images in RGB format.
image = cv2.cvtColor(cv2.imread(filename=str(image_filename)), code=cv2.COLOR_BGR2RGB)

# Resize to MobileNet image shape.
input_image = cv2.resize(src=image, dsize=(224, 224))

input_image = np.expand_dims(input_image.transpose(2, 0, 1), 0)
input_image.shape

result_infer = compiled_model([input_image])[output_layer]
result_index = np.argmax(result_infer)

result_index

In [ ]:
%pwd